In [5]:
pip install cv2

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [7]:
import cv2
import numpy as np
import pandas as pd
import sklearn
from random import randint
import matplotlib.pyplot as plt
import os
import SimpleITK as sitk
import h5py
from skimage.transform import resize, rescale
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import StratifiedKFold


In [8]:
mypath1 = 'D:/19_Papers/Database/test/light/'
output_path = 'D:/19_Papers/Database/test/light1/'
files1 = [f for f in listdir(mypath1) if isfile(join(mypath1, f))]

##Read Input Image
for fname1 in sorted(files1):
    try:
        itk_image = cv2.imread(mypath1+fname1)
        image = np.asarray(itk_image)
        image = resize(image, (224,224), preserve_range= np.int16, order=1)

        image =np.asarray(image)
        cv2.imwrite(output_path+"{}".format(fname1), image)
            
    except RuntimeError:
        pass



In [ ]:
mypath1 = 'D:/COPD/data/EX/cement_raw/'
output_path = 'D:/COPD/data/IN/EX/'
files1 = [f for f in listdir(mypath1) if isfile(join(mypath1, f))]

##Read Input Image
for fname1 in sorted(files1):
    try:
        itk_image = sitk.ReadImage(mypath1+fname1)
        image = sitk.GetArrayFromImage(itk_image)
        image = resize(image, (256,256,256), preserve_range= np.int16, order=1)

        image = sitk.GetImageFromArray(image)
        sitk.WriteImage(image,output_path+"{}".format(fname1))
            
    except RuntimeError:
        pass

In [6]:
mypath1 = 'D:/COPD/data/EX/normal_raw/'
output_path = 'D:/COPD/data/EX/normal1/'
files1 = [f for f in listdir(mypath1) if isfile(join(mypath1, f))]

##Read Input Image
for fname1 in sorted(files1):
    try:
        itk_image = sitk.ReadImage(mypath1+fname1)
        image = sitk.GetArrayFromImage(itk_image)
        image = resize(image, (64,64,64), preserve_range= np.int16, order=1)

        image = sitk.GetImageFromArray(image)
        sitk.WriteImage(image,output_path+"{}".format(fname1))
            
    except RuntimeError:
        pass

In [3]:
############ Read 2D image
mypath1 = 'D:/Ultrasound_Rotator/MuscleData/END3/train/tear3/'
# mypath1 = 'D:/COPD/data/IN/NORMAL256/'
files1 = [f for f in listdir(mypath1) if isfile(join(mypath1, f))]

##Read Input Image
CNN_data = []
for fname1 in sorted(files1):
    try:
        itk_image = sitk.ReadImage(mypath1+fname1)
        image = sitk.GetArrayFromImage(itk_image)
        CNN_data.append(image)
    except RuntimeError:
        pass
    
print('CNN Image Shape : {}'.format(np.shape(CNN_data)))

CNN Image Shape : (140, 224, 224, 3)


In [12]:
COVID = pd.read_csv("D:/Ultrasound_Rotator/MuscleData/tear3.csv")
COVID_labels = COVID.pop('Endpoint')
COVID_labels=COVID_labels.values
CNN_labels = []

print("Befor COVID labels shape is {}".format(COVID_labels.shape))

for i in range(COVID_labels.shape[0]):
    for j in range(3):
        CNN_labels.append(COVID_labels[i])

print("After COVID labels shape is {}".format(np.shape(CNN_labels)))

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)

Befor COVID labels shape is (405,)
After COVID labels shape is (1215,)


In [13]:
######## Image Data & Lables save for learning

with h5py.File('D:/final/Data_Lung_Segment_256.hdf5', 'w') as f:
        data_CNN = f.create_group("Image_data")
        data_CNN.create_dataset("Image",data = CNN_data)
        labels_CNN = f.create_group("Labels_data")
        labels_CNN.create_dataset("Labels", data=CNN_labels)
        f.close()